# Get the necessary packages

In [1]:
!git clone https://github.com/asellam/sitrex.git
%cd sitrex

fatal: destination path 'sitrex' already exists and is not an empty directory.
/content/sitrex


In [2]:
!pip install -r requirements.txt

# Download the dataset

In [3]:
import kagglehub

path = kagglehub.dataset_download('abdellah213/sitrex-dataset')

100%|██████████| 418M/418M [00:03<00:00, 133MB/s]

Extracting files...


# Imports and Setup

In [4]:
# Import necessary libraries
import os
from sklearn.model_selection import train_test_split
from sitrex.preprocessing import load_dataset, preprocess_data, SimilarityDataset, UsefulnessDataset
from sitrex.model import angle_usefulness_model, angle_similarity_model, TQDMProgressBar
import tensorflow as tf

import numpy as np
# Define constants
DATASET_PATH = os.path.join(path, 'dataset')
# EarlyStop save path for the Siamese Transformer
MODEL_SAVE_PATH_SIMILARITY = './angle_similarity_model.keras'
# EarlyStop save path for the Angle Usefulness Transformer
MODEL_SAVE_PATH_USEFULNESS = './angle_usefulness_model.keras'
# Maximum sequence length (L)
MAXLEN = 100
# Batch Size
batch_size = 32

# Loading the data

In [5]:
all_sequences, labels, exercise_angles = load_dataset(DATASET_PATH)

Loading dataset: 100%|██████████| 727/727 [04:32<00:00,  2.67it/s]

Loaded 727 sequences with labels.


# Preprocessing

In [6]:
processed_sequences, numerical_labels, label_angles = preprocess_data(all_sequences, labels, exercise_angles)

# Tensorflow Dataset and Model Routines

# K-Fold Cross-Validation

In [7]:
def split_by_fold(data_x, data_y, folds, test):
    train_x, test_x, train_y, test_y = [], [], [], []
    for fold, split in enumerate(folds):
        for sample in split:
            if fold == test:
                test_x.append(data_x[sample])
                test_y.append(data_y[sample])
            else:
                train_x.append(data_x[sample])
                train_y.append(data_y[sample])
    return train_x, test_x, train_y, test_y

In [8]:
# Preparing the 5 folds
k = 5
N = len(numerical_labels)
indexes = np.arange(N)
np.random.seed(42)
np.random.shuffle(indexes)

fold_size = int(np.ceil(N / k))

folds = []
for i in range(k):
    start = i * fold_size
    end = min(start + fold_size, N)
    folds.append(indexes[start:end])

print(folds)

[array([513, 693, 377,  33,  63, 467, 346, 511, 148, 388, 174,  65, 469,
       428, 707, 350, 210,  72, 449,  78,  54,  39,  97, 211,  81, 557,
       629, 443, 514,  10, 361, 319, 676, 673, 231, 363, 158, 462, 296,
       578, 723, 396,  86, 329, 597, 235, 351, 568, 299, 685, 165, 571,
       164, 223, 539, 399, 281, 227, 714, 620, 199, 705, 155,  49, 332,
       101, 477, 234, 437, 196,  23, 266,  77, 212, 198, 109, 713, 327,
       528, 336, 209,  30, 602, 674, 480, 554, 393, 547, 118, 609, 133,
       688,  84,  79, 213, 433, 448, 181,  31, 254, 598, 215, 412, 591,
       275,  55, 679, 594, 691, 300,  76,   2, 506, 464, 192,  60, 722,
       120, 362, 429, 314, 218, 220, 634, 260, 405, 244, 426, 431,  90,
       331, 538,  69, 204, 131,  44,  70, 349, 717, 569, 135, 601, 239,
       579, 642, 432]), array([136, 286,   6, 657, 587, 334, 250, 145, 338, 132, 306, 660,  41,
       108, 292,  56, 417, 690, 333, 537,  24, 404, 465, 666, 311, 457,
       110,  82,  51, 394, 582, 497, 29

In [9]:
for seq_module in ['gru', 'lstm', 'transformer']:
    print(f'Performing k-fold cross-validation for the Angle Usefulness {seq_module} ...')
    for test in range(k):
        print(f'Testing on Fold #{test+1}/{k} ...')
        X_train, X_test, y_train, y_test = split_by_fold(
            processed_sequences,
            numerical_labels,
            folds,
            test
        )

        X_train, X_val, y_train, y_val = train_test_split(
            X_train,
            y_train,
            test_size=0.2,
            random_state=42
        )

        y_train = np.array(y_train, dtype=np.int32)
        y_test = np.array(y_test, dtype=np.int32)
        y_val = np.array(y_val, dtype=np.int32)

        Y_train = np.empty((len(X_train), len(X_train[0][0])), np.float32)
        Y_val = np.empty((len(X_val), len(X_train[0][0])), np.float32)
        Y_test = np.empty((len(X_test), len(X_train[0][0])), np.float32)

        for (Y, y) in [(Y_train, y_train), (Y_val, y_val), (Y_test, y_test)]:
            for i in range(Y.shape[0]):
                for angle in range(Y.shape[1]):
                    if angle in label_angles[y[i]]:
                        Y[i, angle] = 1
                    else:
                        Y[i, angle] = 0

        usefulness_model = angle_usefulness_model(maxlen=MAXLEN, module=seq_module, lr=1e-3)

        pbar_callback = TQDMProgressBar(epochs=80)

        callback = tf.keras.callbacks.ModelCheckpoint(
            MODEL_SAVE_PATH_USEFULNESS,
            monitor="val_binary_accuracy",
            verbose=0,
            save_best_only=True,
            save_weights_only=False,
            mode="max",
            save_freq="epoch",
            initial_value_threshold=None,
        )

        train_ds = UsefulnessDataset(X_train, Y_train, y_train, batch_size=batch_size, maxlen=MAXLEN, train=True)
        val_ds = UsefulnessDataset(X_val, Y_val, y_val, batch_size=min(batch_size, len(X_val)), maxlen=MAXLEN, train=False)

        history = usefulness_model.fit(
            train_ds,
            validation_data=val_ds,
            batch_size=batch_size,
            epochs=80,
            callbacks=[callback, pbar_callback],
            verbose=0, # Change this to 1 if you want progressive display while training
        )

        # Load the trained classifier model
        usefulness_model = tf.keras.models.load_model(
            MODEL_SAVE_PATH_USEFULNESS
        )

        test_ds = UsefulnessDataset(X_test, Y_test, y_test, batch_size=batch_size, maxlen=MAXLEN, train=False)
        usefulness_model.evaluate(test_ds, verbose=1)

Performing k-fold cross-validation for the Angle Usefulness gru ...
Testing on Fold #1/5 ...


Training Progress: 100%|██████████| 80/80 [03:29<00:00,  2.62s/epoch, loss=0.1861, binary_accuracy=0.9259, precision=0.8677, recall=0.8081, val_loss=0.3080, val_binary_accuracy=0.8637, val_precision=0.7800, val_recall=0.6649]


4/4 [==============================] - 1s 23ms/step - loss: 0.2635 - binary_accuracy: 0.8910 - precision: 0.8116 - recall: 0.7382
Testing on Fold #2/5 ...


Training Progress: 100%|██████████| 80/80 [03:33<00:00,  2.67s/epoch, loss=0.1687, binary_accuracy=0.9310, precision=0.8851, recall=0.8125, val_loss=0.2144, val_binary_accuracy=0.9049, val_precision=0.8485, val_recall=0.7491]


4/4 [==============================] - 1s 24ms/step - loss: 0.2917 - binary_accuracy: 0.8733 - precision: 0.8277 - recall: 0.6590
Testing on Fold #3/5 ...


...

4/4 [==============================] - 1s 39ms/step - loss: 0.2330 - binary_accuracy: 0.9046 - precision: 0.8546 - recall: 0.7781
Testing on Fold #4/5 ...


Training Progress: 100%|██████████| 80/80 [05:30<00:00,  4.14s/epoch, loss=0.1325, binary_accuracy=0.9487, precision=0.9054, recall=0.8730, val_loss=0.2709, val_binary_accuracy=0.8976, val_precision=0.8371, val_recall=0.7628]


4/4 [==============================] - 1s 44ms/step - loss: 0.2894 - binary_accuracy: 0.8801 - precision: 0.8096 - recall: 0.7342
Testing on Fold #5/5 ...


Training Progress: 100%|██████████| 80/80 [06:15<00:00,  4.69s/epoch, loss=0.1385, binary_accuracy=0.9465, precision=0.9051, recall=0.8627, val_loss=0.2062, val_binary_accuracy=0.9108, val_precision=0.8490, val_recall=0.7830]


4/4 [==============================] - 0s 40ms/step - loss: 0.2510 - binary_accuracy: 0.8947 - precision: 0.7915 - recall: 0.8039


In [ ]:
for seq_module in ['gru', 'lstm', 'transformer']:
    print(f'Performing k-fold cross-validation for the Siamese {seq_module} ...')
    for test in range(k):
        print(f'Testing on Fold #{test+1}/{k} ...')
        X_train, X_test, y_train, y_test = split_by_fold(
            processed_sequences,
            numerical_labels,
            folds,
            test
        )

        X_train, X_val, y_train, y_val = train_test_split(
            X_train,
            y_train,
            test_size=0.2,
            random_state=42
        )

        y_train = np.array(y_train, dtype=np.int32)
        y_test = np.array(y_test, dtype=np.int32)
        y_val = np.array(y_val, dtype=np.int32)

        train_batches = 10 * len(X_train) // batch_size
        val_batches = 10 * len(X_val) // batch_size
        train_ds = SimilarityDataset(X_train, y_train, num_batches=train_batches, batch_size=batch_size, label_angles=label_angles, maxlen=MAXLEN, train=True)
        val_ds = SimilarityDataset(X_val, y_val, num_batches=val_batches, batch_size=batch_size, label_angles=label_angles, maxlen=MAXLEN, train=False)

        callback = tf.keras.callbacks.ModelCheckpoint(
            MODEL_SAVE_PATH_SIMILARITY,
            monitor="val_binary_accuracy",
            verbose=0,
            save_best_only=True,
            save_weights_only=False,
            mode="max",
            save_freq="epoch",
            initial_value_threshold=None,
        )

        pbar_callback = TQDMProgressBar(epochs=60)

        # Build and train Siamese model
        similarity_model = angle_similarity_model(maxlen=MAXLEN, module=seq_module, lr=1e-3)
        history = similarity_model.fit(
            train_ds,
            validation_data=val_ds,
            epochs=60,
            callbacks=[callback, pbar_callback],
            verbose=0,  # Change this to 1 if you want progressive display while training
        )

        similarity_model = tf.keras.models.load_model(
            MODEL_SAVE_PATH_SIMILARITY,
        )

        # Make a large dataset in terms of pair in order to have a more reliable test
        # since we can't test on all possible pairs (very huge number)
        test_batches = 1000 * len(X_test) // batch_size
        # Evaluate the Siamese model on the test set
        test_ds = SimilarityDataset(X_test, y_test, num_batches=test_batches, batch_size=batch_size, label_angles=label_angles, maxlen=MAXLEN, train=False)
        similarity_model.evaluate(test_ds, verbose=1)

Performing k-fold cross-validation for the Siamese gru ...
Testing on Fold #1/5 ...


Training Progress: 100%|██████████| 60/60 [20:35<00:00, 20.59s/epoch, loss=0.1674, binary_accuracy=0.9312, precision=0.8821, recall=0.8190, val_loss=0.2420, val_binary_accuracy=0.9007, val_precision=0.8257, val_recall=0.7318]


4562/4562 [==============================] - 193s 42ms/step - loss: 0.2254 - binary_accuracy: 0.9076 - precision: 0.8377 - recall: 0.7654
Testing on Fold #2/5 ...


Training Progress: 100%|██████████| 60/60 [20:02<00:00, 20.04s/epoch, loss=0.1567, binary_accuracy=0.9360, precision=0.8935, recall=0.8257, val_loss=0.2479, val_binary_accuracy=0.8949, val_precision=0.8270, val_recall=0.7089]


4562/4562 [==============================] - 196s 43ms/step - loss: 0.2391 - binary_accuracy: 0.9029 - precision: 0.8449 - recall: 0.7191
Testing on Fold #3/5 ...


...

Training Progress: 100%|██████████| 60/60 [22:22<00:00, 22.38s/epoch, loss=0.1483, binary_accuracy=0.9408, precision=0.9037, recall=0.8389, val_loss=0.2336, val_binary_accuracy=0.9089, val_precision=0.8387, val_recall=0.7581]


4562/4562 [==============================] - 210s 46ms/step - loss: 0.2172 - binary_accuracy: 0.9137 - precision: 0.8578 - recall: 0.7590
Testing on Fold #3/5 ...


Training Progress: 100%|██████████| 60/60 [21:14<00:00, 21.25s/epoch, loss=0.1617, binary_accuracy=0.9336, precision=0.8890, recall=0.8187, val_loss=0.2507, val_binary_accuracy=0.8968, val_precision=0.8015, val_recall=0.7399]


4562/4562 [==============================] - 206s 45ms/step - loss: 0.2392 - binary_accuracy: 0.9036 - precision: 0.8292 - recall: 0.7556
Testing on Fold #4/5 ...


Training Progress:  52%|█████▏    | 31/60 [10:45<09:42, 20.08s/epoch, loss=0.2123, binary_accuracy=0.9121, precision=0.8603, recall=0.7475, val_loss=0.2446, val_binary_accuracy=0.9030, val_precision=0.8404, val_recall=0.7348]

# One-shot Generalization Evaluation

In [ ]:
tests = [[3, 4, 13, 14, 15, 17], [1, 3, 4, 13, 14, 20], [1, 3, 6, 13, 19, 20], [1, 3, 7, 13, 14, 16], [1, 3, 5, 13, 14, 19], [1, 2, 3, 13, 19, 21], [0, 3, 6, 7, 13, 19], [1, 3, 4, 13, 14, 21], [3, 7, 13, 14, 16, 17], [1, 3, 10, 13, 20, 21]]

In [ ]:
# Function to split data by specifying test labels
def split_by_labels(data_x, data_y, test_labels):
    X_train, X_test, y_train, y_test = [], [], [], []
    for sample_x, sample_y in zip(data_x, data_y):
        if sample_y in test_labels:
            X_test.append(sample_x)
            y_test.append(sample_y)
        else:
            X_train.append(sample_x)
            y_train.append(sample_y)
    return X_train, X_test, y_train, y_test

In [ ]:
print('Performing one-shot validation for the Angle Usefulness Transformer ...')
for test in tests:
    print(f'Testing on Exercises: {test} ...')
    X_train, X_test, y_train, y_test = split_by_labels(
        processed_sequences,
        numerical_labels,
        test_labels=test,
    )

    X_train, X_val, y_train, y_val = train_test_split(
        X_train,
        y_train,
        test_size=0.2,
        random_state=42
    )

    y_train = np.array(y_train, dtype=np.int32)
    y_test = np.array(y_test, dtype=np.int32)
    y_val = np.array(y_val, dtype=np.int32)

    Y_train = np.empty((len(X_train), len(X_train[0][0])), np.float32)
    Y_val = np.empty((len(X_val), len(X_train[0][0])), np.float32)
    Y_test = np.empty((len(X_test), len(X_train[0][0])), np.float32)

    for (Y, y) in [(Y_train, y_train), (Y_val, y_val), (Y_test, y_test)]:
        for i in range(Y.shape[0]):
            for angle in range(Y.shape[1]):
                if angle in label_angles[y[i]]:
                    Y[i, angle] = 1
                else:
                    Y[i, angle] = 0

    usefulness_model = angle_usefulness_model(maxlen=MAXLEN, module='transformer', lr=1e-4)

    callback = tf.keras.callbacks.ModelCheckpoint(
        MODEL_SAVE_PATH_USEFULNESS,
        monitor="val_binary_accuracy",
        verbose=0,
        save_best_only=True,
        save_weights_only=False,
        mode="max",
        save_freq="epoch",
        initial_value_threshold=None,
    )

    pbar_callback = TQDMProgressBar(epochs=80)

    train_ds = UsefulnessDataset(X_train, Y_train, y_train, batch_size=batch_size, maxlen=MAXLEN, train=True)
    val_ds = UsefulnessDataset(X_val, Y_val, y_val, batch_size=min(batch_size, len(X_val)), maxlen=MAXLEN, train=False)

    history = usefulness_model.fit(
        train_ds,
        validation_data=val_ds,
        batch_size=batch_size,
        epochs=80,
        callbacks=[callback, pbar_callback],
        verbose=0,  # Change this to 1 if you want progressive display while training
    )

    # Load the trained classifier model
    usefulness_model = tf.keras.models.load_model(
        MODEL_SAVE_PATH_USEFULNESS
    )

    test_ds = UsefulnessDataset(X_test, Y_test, y_test, batch_size=batch_size, maxlen=MAXLEN, train=False)
    usefulness_model.evaluate(test_ds, verbose=1)

In [ ]:
print('Performing one-shot validation for the Siamese Transformer ...')
for test in tests:
    print(f'Testing on Exercises: {test} ...')

    X_train, X_test, y_train, y_test = split_by_labels(
        processed_sequences,
        numerical_labels,
        test_labels=test,
    )

    X_train, X_val, y_train, y_val = train_test_split(
        X_train,
        y_train,
        test_size=0.2,
        random_state=42
    )
    train_batches = 10 * len(X_train) // batch_size
    val_batches = 10 * len(X_val) // batch_size
    train_ds = SimilarityDataset(X_train, y_train, num_batches=train_batches, batch_size=batch_size, label_angles=label_angles, maxlen=MAXLEN, train=True)
    val_ds = SimilarityDataset(X_val, y_val, num_batches=val_batches, batch_size=batch_size, label_angles=label_angles, maxlen=MAXLEN, train=False)

    callback = tf.keras.callbacks.ModelCheckpoint(
        MODEL_SAVE_PATH_SIMILARITY,
        monitor="val_binary_accuracy",
        verbose=0,
        save_best_only=True,
        save_weights_only=False,
        mode="max",
        save_freq="epoch",
        initial_value_threshold=None,
    )

    pbar_callback = TQDMProgressBar(epochs=60)

    # Build and train Siamese model
    similarity_model = angle_similarity_model(maxlen=MAXLEN, module='transformer', lr=1e-3)
    history = similarity_model.fit(
        train_ds,
        validation_data=val_ds,
        epochs=60,
        callbacks=[callback, pbar_callback],
        verbose=0,  # Change this to 1 if you want progressive display while training
    )

    # Load the trained Siamese model
    similarity_model = tf.keras.models.load_model(
        MODEL_SAVE_PATH_SIMILARITY,
    )

    # Make a large dataset in terms of pair in order to have a more reliable test
    # since we can't test on all possible pairs (very huge number)
    test_batches = 1000 * len(X_test) // batch_size
    # Evaluate the Siamese model on the test set
    test_ds = SimilarityDataset(X_test, y_test, num_batches=test_batches, batch_size=batch_size, label_angles=label_angles, maxlen=MAXLEN, train=False)
    similarity_model.evaluate(test_ds, verbose=1)